# Julia 機器學習：DecisionTree 決策樹

## 作業 030：乳癌預測資料集

請使用隨機森林模型建立一個分類模型，來預測乳癌資料集中，為良性或是惡性的腫瘤。

In [2]:
using Pkg

In [3]:
Pkg.add("DecisionTree")

    Updating registry at `C:\Users\evanl\.julia\registries\General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Resolving package versions...
   Installed DecisionTree ─ v0.10.10
    Updating `C:\Users\evanl\.julia\environments\v1.6\Project.toml`
  [7806a523] + DecisionTree v0.10.10
    Updating `C:\Users\evanl\.julia\environments\v1.6\Manifest.toml`
  [7806a523] + DecisionTree v0.10.10
Precompiling project...
  ✓ DecisionTree
  1 dependency successfully precompiled in 5 seconds (215 already precompiled)


In [4]:
Pkg.add("ScikitLearn")

   Resolving package versions...
   Installed ScikitLearn ─ v0.6.4
    Updating `C:\Users\evanl\.julia\environments\v1.6\Project.toml`
  [3646fa90] + ScikitLearn v0.6.4
    Updating `C:\Users\evanl\.julia\environments\v1.6\Manifest.toml`
  [d96e819e] + Parameters v0.12.3
  [3646fa90] + ScikitLearn v0.6.4
  [3a884ed6] + UnPack v1.0.2
Precompiling project...
  ✓ ScikitLearn
  1 dependency successfully precompiled in 7 seconds (218 already precompiled)


In [5]:
using DecisionTree, RDatasets, DataFrames, MLDataUtils, Statistics

## 讀取資料

In [6]:
biopsy = dataset("MASS", "biopsy")

,ID,V1,V2,V3,V4,V5,V6,V7,V8,V9,Class
,String,Int32,Int32,Int32,Int32,Int32,Int32?,Int32,Int32,Int32,Cat…
1,1000025,5,1,1,1,2,1,3,1,1,benign
2,1002945,5,4,4,5,7,10,3,2,1,benign
3,1015425,3,1,1,1,2,2,3,1,1,benign
4,1016277,6,8,8,1,3,4,3,7,1,benign
5,1017023,4,1,1,3,2,1,3,1,1,benign
6,1017122,8,10,10,8,7,10,9,7,1,malignant
7,1018099,1,1,1,1,2,10,3,1,1,benign
8,1018561,2,1,2,1,2,1,3,1,1,benign
9,1033078,2,1,1,1,2,1,1,1,5,benign


In [7]:
biopsy = dropmissing(biopsy)

,ID,V1,V2,V3,V4,V5,V6,V7,V8,V9,Class
,String,Int32,Int32,Int32,Int32,Int32,Int32,Int32,Int32,Int32,Cat…
1,1000025,5,1,1,1,2,1,3,1,1,benign
2,1002945,5,4,4,5,7,10,3,2,1,benign
3,1015425,3,1,1,1,2,2,3,1,1,benign
4,1016277,6,8,8,1,3,4,3,7,1,benign
5,1017023,4,1,1,3,2,1,3,1,1,benign
6,1017122,8,10,10,8,7,10,9,7,1,malignant
7,1018099,1,1,1,1,2,10,3,1,1,benign
8,1018561,2,1,2,1,2,1,3,1,1,benign
9,1033078,2,1,1,1,2,1,1,1,5,benign


In [29]:
train_x = Matrix(biopsy[1:500,2:10])
test_x = Matrix(biopsy[501:683,2:10])
train_y = string.(biopsy[1:500,11])
test_y = string.(biopsy[501:683,11])

183-element Vector{String}:
 "malignant"
 "benign"
 "benign"
 "benign"
 "malignant"
 "benign"
 "benign"
 "malignant"
 "malignant"
 "benign"
 "benign"
 "benign"
 "benign"
 ⋮
 "benign"
 "benign"
 "benign"
 "benign"
 "malignant"
 "benign"
 "benign"
 "benign"
 "benign"
 "malignant"
 "malignant"
 "malignant"

In [30]:
model = DecisionTree.DecisionTreeClassifier(max_depth=2)

DecisionTreeClassifier
max_depth:                2
min_samples_leaf:         1
min_samples_split:        2
min_purity_increase:      0.0
pruning_purity_threshold: 1.0
n_subfeatures:            0
classes:                  nothing
root:                     nothing

In [31]:
DecisionTree.fit!(model,train_x,train_y)

DecisionTreeClassifier
max_depth:                2
min_samples_leaf:         1
min_samples_split:        2
min_purity_increase:      0.0
pruning_purity_threshold: 1.0
n_subfeatures:            0
classes:                  ["benign", "malignant"]
root:                     Decision Tree
Leaves: 4
Depth:  2

In [32]:
predict=DecisionTree.predict(model, test_x)

183-element Vector{String}:
 "malignant"
 "benign"
 "benign"
 "benign"
 "malignant"
 "benign"
 "benign"
 "malignant"
 "malignant"
 "benign"
 "benign"
 "benign"
 "benign"
 ⋮
 "benign"
 "benign"
 "benign"
 "benign"
 "malignant"
 "benign"
 "benign"
 "benign"
 "benign"
 "malignant"
 "malignant"
 "malignant"

In [33]:
accuracy() = mean((predict .== test_y))*100
accuracy()

95.08196721311475